In [8]:
from astroquery.simbad import Simbad
import pandas as pd

In [9]:
Simbad.ROW_LIMIT = 1000 # now any query fetches at most 15 rows for testing purposes
Simbad.reset_votable_fields()
Simbad.add_votable_fields("distance", "flux(K)", "flux(G)", "flux(R)", "flux(B)")

In [10]:
test_table = Simbad.query_criteria(maintype="RR?")

In [11]:
df = test_table.to_pandas()
df.to_pickle('./RRL_symbad.pkl')

# Scrapping the Christine Clement database of variable stars

This database has a comprehensive list of a lot of pulsating stars in different clusters with their type and magnitude listed. We are going to extract all the stars in the different clusters, and name them in the following manner:

`Cluster_IDnumber`

We will then try to find their ID in the SIMBAD database. Hopefully there is some database out there that has the names of the different stars in a cluster, but if it does not exist we will attempt to use the coordinates of the star to find it in SIMBAD.

In [12]:
from selenium import webdriver

In [13]:
driver = 'C:/Users/cuco2/OneDrive/Escritorio/Physics/Year 4/Final year project/MphysProject/Drivers/chromedriver.exe'
options = webdriver.ChromeOptions()
# options.add_argument('headless')

driver = webdriver.Chrome(executable_path = driver, options = options)

In [14]:
url = "http://www.astro.utoronto.ca/~cclement/cat/listngc.html"
driver.get(url)

In [15]:
cluster_list_imported = driver.find_elements_by_xpath("//a[@href]")
cluster_list_cleaned = [elem.get_attribute("href") for elem in cluster_list_imported]

In [16]:
# We're removing the list with the last updates
cluster_list_cleaned = cluster_list_cleaned[1:]

In [17]:
# Create a function to extract the data from each of the cluster subpages

import re
from io import StringIO

def extract_cluster_data(text):
    # Create a dictionary to get rid of all the equal signs (=) as line breaks
    dict_list = re.split(r"=", text)
    dict_list = list(dict.fromkeys(dict_list))

    # The actual stars are the 8th element of the list
    # Save as StringIO so we can put it in a df later
    data = StringIO(dict_list[7])

    # This contains the information in each column of the table, the 6th element in the list
    header = dict_list[6]
    header = header.split(" ")
    header = list(dict.fromkeys(header))
    header.remove('')

    df = pd.read_csv(data, sep = "\s+", names = header)

    # Obtain the clusterName, the first element in the list
    clusterName = list(re.findall(">.*\n*", dict_list[3])[0])
    clusterName.remove('>')
    clusterName.remove('\n')
    clusterName = ''.join(clusterName)

    # Rename the ID column so each one is called clusterName_number
    for i in range(len(df["\nID"])):
        df.loc[:,("\nID", i)] = f"{clusterName}_{i+1}"

    return df

In [18]:
# create an empty df

all_variable_stars = pd.DataFrame()

for url in cluster_list_cleaned[3:]:
    driver.get(url)
    cluster_data = driver.page_source
    variable_stars = extract_cluster_data(cluster_data)
    break

ParserError: Error tokenizing data. C error: Expected 12 fields in line 3, saw 13


In [ ]:
cluster_list_cleaned[3]

In [ ]:
variable_stars

In [ ]:
for url in cluster_list_cleaned:
    print(driver.get(url))


In [ ]:


mylist = re.split(r"=", h)
mylist = list(dict.fromkeys(mylist))
data = StringIO(mylist[7])

header = mylist[6]
header = header.split(" ")
header = list(dict.fromkeys(header))
header.remove('')

df = pd.read_csv(data, sep = "\s+", names = header)
df

In [ ]:
clusterName = list(re.findall(">.*\n*", mylist[3])[0])
clusterName.remove('>')
clusterName.remove('\n')
clusterName = ''.join(clusterName)

In [ ]:
for i in range(len(df["\nID"])):
    df["\nID"].loc[i] = f"{clusterName}_{i+1}"